In [19]:
import torch
import numpy as np

from torch import optim, nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset


import torch
from torch.utils.data import random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as td
import torchvision.transforms as transforms
import torchvision.datasets as datasets


def custom_loader(batch_size, shuffle_test=False, data_path='./Dataset/Train'):
    # Add the necessary transforms
    # normalize = transforms.Normalize(mean=[0.024], std=[0.994])
    transform = transforms.Compose([
        transforms.Resize((48, 48)),  # Adjust this if your images are a different size
        # transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
        transforms.ToTensor(),
        # normalize
    ])

    # Load your dataset using ImageFolder
    master_dataset = datasets.ImageFolder(root=data_path, transform=transform)

    # Calculate the sizes of the splits
    total_size = len(master_dataset)
    train_size = int(0.85 * total_size)
    val_size = total_size - train_size
    # print(val_size)
    # print(train_size)

    # Use random_split to create datasets for training, testing, and validation
    train_dataset, val_dataset = random_split(master_dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader



class MultiLayerFCNet(nn.Module):
    def __init__(self,input_size, hidden_size, output_size):
        super().__init__()

        self.layer1=nn.Conv2d(3,32,3,padding=1,stride=1)
        self.B1 = nn.BatchNorm2d(32)
        self.layer2 = nn.Conv2d(32, 32, 3, padding=1, stride=1)
        self.B2 = nn.BatchNorm2d(32)
        self.Maxpool=nn.MaxPool2d(2)
        self.layer3 = nn.Conv2d(32, 64, 3, padding=1, stride=1)
        self.B3 = nn.BatchNorm2d(64)
        self.layer4 = nn.Conv2d(64, 64, 3, padding=1, stride=1)
        self.B4 = nn.BatchNorm2d(64)
        self.dropout = nn.Dropout(0.5)
        
        # New layers
        # self.layer5 = nn.Conv2d(64, 128, 4, padding=1, stride=1)
        # self.B5 = nn.BatchNorm2d(128)
        # self.layer6 = nn.Conv2d(128, 128, 4, padding=1, stride=1)
        # self.B6 = nn.BatchNorm2d(128)
        # self.layer7 = nn.Conv2d(128, 256, 4, padding=1, stride=1)
        # self.B7 = nn.BatchNorm2d(256)
        # self.layer8 = nn.Conv2d(256, 256, 4, padding=1, stride=1)
        # self.B8 = nn.BatchNorm2d(256)
        
        # Calculate the size for the fully connected layer after additional max-pooling layers
        # Assuming two max-pooling operations in the existing layers
        self.fc_size = 64 * 12 * 12   # Now this is 256 * 3 * 3
        self.fc = nn.Linear(self.fc_size, output_size)

    def forward(self, x):
        # Pass through existing layers
        x = F.leaky_relu(self.B1(self.layer1(x)))
        x = self.Maxpool(F.leaky_relu(self.B2(self.layer2(x))))
        x = F.leaky_relu(self.B3(self.layer3(x)))
        x = self.Maxpool(F.leaky_relu(self.B4(self.layer4(x))))
        

        #print(x.shape)  # Add this line to check the shape
        # Pass through new layers
        # x = F.leaky_relu(self.B5(self.layer5(x)))
        # x = F.leaky_relu(self.B6(self.layer6(x)))
        # x = self.Maxpool(F.leaky_relu(self.B7(self.layer7(x))))
        # x = self.Maxpool(F.leaky_relu(self.B8(self.layer8(x))))
        
        x = x.view(x.size(0), -1)  # Flatten the tensor for the fully connected layer
        return self.fc(x)

if __name__ == '__main__':

    batch_size = 64
    test_batch_size = 64
    input_size = 3 * 48 * 48  # 1 channels, 48x48 image size
    hidden_size = 50  # Number of hidden units
    output_size = 4  # Number of output classes 4
    num_epochs = 10

    # train_loader, _ = cifar_loader(batch_size)
    # _, test_loader = cifar_loader(test_batch_size)
    train_loader, test_loader = custom_loader(batch_size, data_path='./dataset/Train')
    # dataloader = DataLoader(dataset=IrisDataset('iris.data'),
    #                         batch_size=10,
    #                         shuffle=True)

    epochs = 50
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = MultiLayerFCNet(input_size, hidden_size, output_size)
    model = nn.DataParallel(model)
    model.to(device)
    #model.load_state_dict(torch.load('path'))

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    BestACC=0.3
    for epoch in range(epochs):
        running_loss = 0
        for instances, labels in train_loader:
            optimizer.zero_grad()

            output = model(instances)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(running_loss / len(train_loader))

        model.eval()
        with torch.no_grad():
            allsamps=0
            rightPred=0

            for instances, labels in test_loader:

                output = model(instances)
                predictedClass=torch.max(output,1)
                allsamps+=output.size(0)
                rightPred+=(torch.max(output,1)[1]==labels).sum()


            ACC=rightPred/allsamps
            print("epoch=",epoch)
            print('Accuracy is=',ACC*100)
            #if the acc is greater than the best acc, save the model
            
            if ACC>BestACC:
                torch.save(model.state_dict(), './model/model_variant1.pth')
                BestACC=ACC

            #save the model architecture

            # torch.save(model, './model/model.pth')
            # torch.save(model.state_dict(), './model/model_state_dict.pth')
            # torch.save({
            #     'model': model.state_dict(),
            #     'optimizer': optimizer.state_dict(),
            #     'epoch': epoch,
            #     'loss': loss
            # }, './model/all.tar')
        model.train()



9.100479366732579
epoch= 0
Accuracy is= tensor(39.6491)
1.3340063562580184
epoch= 1
Accuracy is= tensor(44.0351)
1.1675117869003147
epoch= 2
Accuracy is= tensor(45.4386)
1.0721565344754387
epoch= 3
Accuracy is= tensor(49.4737)
1.0389226219233345
epoch= 4
Accuracy is= tensor(50.)
0.9315179376041188
epoch= 5
Accuracy is= tensor(54.7368)
0.8684279626491023
epoch= 6
Accuracy is= tensor(54.9123)
0.8186756559446746
epoch= 7
Accuracy is= tensor(58.4211)
0.7531094971825095
epoch= 8
Accuracy is= tensor(57.5439)
0.7188600348491295
epoch= 9
Accuracy is= tensor(55.2632)
0.7086352302747614
epoch= 10
Accuracy is= tensor(58.5965)
0.6225613811436821
epoch= 11
Accuracy is= tensor(55.9649)
0.5825706276239133
epoch= 12
Accuracy is= tensor(61.2281)
0.5386465799574759
epoch= 13
Accuracy is= tensor(61.9298)
0.5353217288559559
epoch= 14
Accuracy is= tensor(57.1930)
0.4446299677970363
epoch= 15
Accuracy is= tensor(63.3333)
0.4075356911794812
epoch= 16
Accuracy is= tensor(64.5614)
0.41992168274580266
epoch= 17